In [132]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scipy.constants as c
from scipy import stats

from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAnnulus

from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS
from astropy import wcs
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy import units as u

import os
import sys
from astropy.table import Table, join, vstack
from astropy.nddata.utils import extract_array
import pyregion
import tqdm 
import argparse

import integratedflux as flux
from importlib import reload
reload(flux)
import warnings
from astropy.utils.exceptions import AstropyWarning

import time

In [133]:
directory_univ_I = r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/I_slices/'
directory_local = r'/data2/goesaert/' 

region_name = "Abell85_catalog_BDSF_rsl.reg"

rsl = fits.open(get_pkg_data_filename(directory_local+'Abell85_catalog_BDSF_rsl.fits'))[1].data

In [140]:
start = time.time()

hdul_image = fits.open(directory_univ_I+'Abell_85_I_plane_freq1.fits')

for n in range(100):
    measurement = flux.integratedflux(directory_local+'Abell_85_I_plane_freq1.fits', mask, rsl, n, hdul=hdul_image)
    measurement = flux.integratedflux(directory_local+'Abell_85_I_plane_freq1.fits', np.invert(mask), rsl, n, hdul=hdul_image)

#uncertainty = flux.uncertainty_flux(I_dir, measurement, beamnumb, rms=0.01*measurement, delta_cal=0.1)
#print(measurement, beamnumb, uncertainty)
end = time.time()
print("The time of execution of above program is :", end-start)

The time of execution of above program is : 8.266081094741821


In [ ]:
for n in range(12):
    

In [97]:
test = np.array([0,1,2,3,4,5,6])
print(np.nansum(test))

21


In [6]:
warnings.filterwarnings('ignore', category=AstropyWarning, append=True)
mask = flux.mask_regions(directory_local+'Abell_85_I_plane_freq1.fits', region_name,'/data2/goesaert/testmask.fits', maskoutside=False)

Writing mask file to /data2/goesaert/testmask.fits
